In [1]:
# 123123456789

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import date,timedelta,datetime
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
# %pip install "azureml-core==1.18.0.post3" "azureml-dataprep==2.6.0"

In [3]:
#　未更動
storage_name = "oselakepseainfradls"
storage_key = "IlXQkTls7o9JJ70c6Oswx/6v6asaaOCm1/6/ckcji7YRcy13ylqXizxgddk0ExTVZwzoV3zp16RtlZDhU9h3Aw=="
storage_url = f"https://{storage_name}.blob.core.windows.net/"

In [4]:
#############
connection_string = 'DefaultEndpointsProtocol=https;AccountName=oselakepseainfradls;AccountKey=Ny6p7hVS+DpEH3N7K20GIt0OoH1t2JuG2jl4fILjlYI7qda9JMDdrUe1SCl/8aMrz/dwLxGg3cEF9U+kQ/jkgA==;BlobEndpoint=https://oselakepseainfradls.blob.core.windows.net/;QueueEndpoint=https://oselakepseainfradls.queue.core.windows.net/;TableEndpoint=https://oselakepseainfradls.table.core.windows.net/;FileEndpoint=https://oselakepseainfradls.file.core.windows.net/;'

blob_client = BlobClient.from_connection_string(
       connection_string, container_name="sap-addon", blob_name="metal_history_price_list_newcol.csv")


In [5]:
properties = blob_client.get_blob_properties()

In [6]:
with open('C:\\Users\\User\\Desktop\\demo\\metal_history_price_list_newcol.csv', "wb") as my_blob:
    download_stream = blob_client.download_blob()
    my_blob.write(download_stream.readall())

In [7]:
# LOCALFILE is the file path
dataframe_blobdata = pd.read_csv('C:\\Users\\User\\Desktop\\demo\\metal_history_price_list_newcol.csv')

In [8]:
def getCopperPrice(date):
    url = 'https://www.cnyes.com/futures/History.aspx?mydate={}&code=CDCS'.format(date)
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find_all('table')
    result_target = result[1].select('td.cr')
    time.sleep(2)
    
    return result_target[3].text

In [9]:
def getMetalData(year):
    url_g_s_p = 'https://www.kitco.com/londonfix/gold.londonfix{}.html'.format(year)
    date_list = []
    gold_list_am = []
    gold_list_pm = []
    silver_list = []
    palladium_list_am = []
    palladium_list_pm = []
    copper_list = []
    
    response = requests.get(url_g_s_p)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Get date list
    results_date = soup.select('td.date',limit=7)
    del results_date[0]   

    for date in results_date:
        datetext = date.text
        date1 = datetext.replace('-0','-')#取代
        date2 = date1.replace('-','/')#取代
        date_list.append(date2)
    
    # Gold list
    results_gold_am = soup.select('td.au.am')
    results_gold_pm = soup.select('td.au.pm')

    del results_gold_am[0]
    del results_gold_pm[0]

    for i in range(len(results_gold_am)):
        if '-' in results_gold_am[i].text :
            gold_price_am = ''
            gold_list_am.append(gold_price_am)
        else:
            gold_price_am = float(results_gold_am[i].text)
            gold_list_am.append(str(gold_price_am))

        if '-' in results_gold_pm[i].text :
            gold_price_pm = ''
            gold_list_pm.append(gold_price_pm)
        else:
            gold_price_pm = float(results_gold_pm[i].text)
            gold_list_pm.append(str(gold_price_pm))
        
    # Silver list
    results_silver = soup.select('td.ag')
    results_silver[0:2] = []
    
    for silver in results_silver:
        silver_list.append(silver.text)
        
    # Palladium list
    results_palladium_am = soup.select('td.pl.am')
    results_palladium_pm = soup.select('td.pl.pm')

    del results_palladium_am[0]
    results_palladium_pm[0:2] = []
    
    for i in range(len(results_palladium_am)):
        if '-' in results_palladium_am[i].text :
            palladium_price_am = ''
            palladium_list_am.append(palladium_price_am)
        else:
            palladium_price_am = float(results_palladium_am[i].text)
            palladium_list_am.append(str(palladium_price_am))
            
        if '-' in results_palladium_pm[i].text :
            palladium_price_pm = ''
            palladium_list_pm.append(palladium_price_pm)
        else:
            palladium_price_pm = float(results_palladium_pm[i].text)
            palladium_list_pm.append(str(palladium_price_pm))  

        
    # Copper list
    for date in results_date:
        copper_list.append(getCopperPrice(date.text.replace('-','')))
        
    header_list = ['Date', 'Gold(am)', 'Gold(pm)', 'Silver', 'Palladium (am)','Palladium (pm)', 'Copper']
    df = pd.DataFrame(list(zip(date_list, gold_list_am, gold_list_pm,silver_list, palladium_list_am,palladium_list_pm, copper_list)), columns=header_list)
    df
    return df

In [10]:
#取得今日日期
today = date.today()
last_day = today-timedelta(days=7)
today_str = today.strftime('%Y-%m-%d')
last_day_str = last_day.strftime('%Y-%m-%d')
year1 = today_str[2:4]
df_all_years = []

if today_str[2:4] == last_day_str[2:4]:
    df = getMetalData(today_str[2:4])
    df_all_years.append(df)
else:
    df = getMetalData(today_str[2:4])
    df_all_years.append(df)
    df = getMetalData(last_day_str[2:4])
    df_all_years.append(df)
    
df_result = pd.concat(df_all_years)
df_result

Out[10]:

,Date,Gold(am),Gold(pm),Silver,Palladium (am),Palladium (pm),Copper
0,2021/1/27,1846.4,1843.0,25.1850,2318.0,2312.0,"7,821.250"
1,2021/1/26,1853.2,1856.6,25.4000,2325.0,2324.0,"8,007.250"
2,2021/1/25,1855.6,1856.85,25.6350,2368.0,2347.0,"7,965.500"
3,2021/1/22,1853.6,1852.7,25.3200,2363.0,2355.0,"7,993.500"
4,2021/1/21,1867.65,1862.1,25.8650,2387.0,2394.0,"8,014.250"
5,2021/1/20,1854.6,1856.6,25.2950,2370.0,2377.0,"8,040.250"


In [11]:
data_all = df_result.append(dataframe_blobdata, ignore_index = True)    #新舊資料合併
data_all.drop_duplicates(subset ="Date",keep = 'first', inplace = True) #刪除重複值

In [12]:
# 回存地端檔案
data_all.to_csv(r'C:\Users\User\Desktop\demo\metal_history_price_list_newcol.csv', index = False)

In [13]:
blob_client = BlobClient.from_connection_string(
       connection_string, container_name="sap-addon", blob_name="metal_history_price_list_newcol.csv")
with open(r'C:\Users\User\Desktop\demo\metal_history_price_list_newcol.csv', "rb") as my_blob:
    blob_client.upload_blob(my_blob, blob_type="BlockBlob",overwrite=True)